In [1]:
import os
import re
import pandas as pd
import text_input
import importlib

## Parameters

In [2]:
report  = "R2"
path2SP = f"/Users/{os.getlogin()}/OneDrive - World Justice Project/EU Subnational/EU-S Data/reports/eu-thematic-reports"

## Reading outline

In [3]:
# Reading outline
outline = pd.read_excel(f"{path2SP}/data-viz/inputs/report_outline.xlsx")
charts4reports = (
    outline.copy()
    .loc[(outline["thematic_reports"] == True)]
)

# Creating text input classes

In [4]:
# Reading text inputs
input_files = [
    "title-&-scroll.md",
    "introduction.md",
    # "executive-summary.md",
    "thematic-findings.md",
    "appendix.md",
    "methodology.md",
    "about.md",
    "acknowledgements.md"
]
inputs_as_classes = [text_input.text_input(report, x) for x in input_files]

## Wrangling and exporting data

In [5]:
versions = {
    "html-version" : False,
    "pdf-version"  : True
}
for version, parameter in versions.items():
    input_data = [x.get_csv(charts4reports, pdfver=parameter) for x in inputs_as_classes]
    front_page = [inputs_as_classes[0].get_front_page()]
    csv_data   = pd.concat(front_page + input_data)
    csv_data.columns = ["id", "tipo de elemento", "contenido (markdown)", "pertenece a ", "settings"]
    csv_data.to_csv(f"{report}/{report}-csv-schema-{version}.csv", index = False, encoding = "utf-8") 